In [1]:
import numpy as np
import pandas as pd
from daty_transakcji import *
from osoby import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [2]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "PESEL",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [3]:
employees_df

,id,imię,nazwisko,PESEL,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Sebastian,Ślesicki,74022464650,Szczecin,ul. Miodowa 78/27,48687303954,sebastian.slesicki@kuba.pl,2014-01-01,None,menadżer,64.3
1,2,Wiesław,Trần,94012709112,Wrocław,ul. Cmentarna 74,48649433059,wieslaw.tran@kuba.pl,2014-01-01,None,sprzedawca,52.8
2,3,Barbara,Kosowska,60032137046,Kielce,ul. Kozienicka 57/18,48575701641,barbara.kosowska@kuba.pl,2014-01-01,None,sprzedawca,56.4
3,4,Bożena,Wencka,64012383504,Wrocław,ul. Walcowników 55/34,48003487800,bozena.wencka@kuba.pl,2014-01-01,2017-02-24,mechanik,45.9
12,5,Anna,Kosobucka,62101631026,Wrocław,ul. Złotopolska 2/17,48196414849,anna.kosobucka@kuba.pl,2014-01-01,None,informatyk,55.8
4,6,Małgorzata,Beliak,76060151943,Wrocław,ul. Długa 17/24,48191553611,malgorzata.beliak@kuba.pl,2014-12-30,None,mechanik,49.7
5,7,Sabina,Naguschewska,69112373905,Wrocław,ul. Owocowa 11,48112334660,sabina.naguschewska@kuba.pl,2015-08-19,None,mechanik,46.3
6,8,Lena,Olejnik,85082378805,Wrocław,ul. Fiołkowa 42,48939102145,lena.olejnik@kuba.pl,2015-10-21,2016-10-20,mechanik,47.7
7,9,Ewa,Sobel,67020910223,Warszawa,ul. Gwiezdna 48,48652438860,ewa.sobel@kuba.pl,2016-01-07,None,pomocnik mechanika,43.7
8,10,Magdalena,Jabłońska,66072400566,Wrocław,ul. Konieczyńskich 13,48064170528,magdalena.jablonska@kuba.pl,2016-03-10,None,pomocnik mechanika,39.2


In [4]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [5]:
no_transactions = np.random.poisson(2000)
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [6]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [7]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4])-19 for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()
not_fixed_our_cars = our_cars[:]

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if len(not_fixed_our_cars) >= 1 and np.random.rand() <= weights[1]/2: #fixing car we currently have
           client_ids[i] = 0
           car_ids[i] = np.random.choice(not_fixed_our_cars)
           not_fixed_our_cars.remove(car_ids[i])
        elif np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [8]:
len(year_ceil), len(pd.unique(client_ids))

(1459, 1460)

In [9]:
year_ceil

[1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [10]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [11]:
len(buy_indexes[0]), len(sell_indexes[0]), len(service_indexes[0])

(748, 741, 501)

In [12]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [13]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [14]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [15]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [16]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])

menager = employees_df.iloc[0]
if int(menager["PESEL"][-2]) % 2 == 1:
    menager_sex = "M"
else:
    menager_sex = "K"
clients_df.loc[0] = [0, menager["imię"], menager["nazwisko"], menager_sex, menager["PESEL"], menager["miasto"], menager["adres"], False, menager["telefon"], menager["email"]]

unique_clients = np.unique(client_ids)
unique_clients = unique_clients[unique_clients != 0]
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Sebastian,Ślesicki,M,74022464650,Szczecin,ul. Miodowa 78/27,False,48687303954,sebastian.slesicki@kuba.pl
1,1,Filip,Maciejewski,M,60060840459,Wrocław,ul. Pułtuska 63/34,False,48210039472,re291kc2b9q4d4eu@opayq.com
2,2,Elena,Gamrat,K,74110877795,Szczecin,ul. Oskara Kolberga 94/2,False,48744588708,e-gamra74@wp.pl
3,3,Victoria,Kuklińska,K,90020124895,Wrocław,ul. Jana Pawła II 18/7,False,48068319561,l8xed0t9129@gmail.com
4,4,Paulina,Lechończak,K,93092394292,Świdnica,ul. Podgórna 14/9,False,48819095045,pau_lechonc8037@onet.pl
...,...,...,...,...,...,...,...,...,...,...
1455,1455,Józef,Gerlipp-Krajewski,M,86050432152,Tomaszów Mazowiecki,ul. Wiązowa 46/28,False,48757553740,gerlip.jozef@wp.pl
1456,1456,Maria,Smułkowska,K,60070730856,Wąchock,ul. Kolibrów 77/7,False,48426326948,smulkows.m@wp.pl
1457,1457,Krystian,Gumula,M,72120117856,Wrocław,ul. Strażacka 70/2,False,48042324185,kry-gumul1972@gmail.com
1458,1458,Maria,Ścisłowicz,K,None,Gdynia,ul. Różana 98/45,False,48524055862,marscislowic@o2.pl


In [17]:
pd.unique(clients_df["id"])

array([   0,    1,    2, ..., 1457, 1458, 1459])

In [18]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [19]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Sebastian,Ślesicki,M,74022464650,Szczecin,ul. Miodowa 78/27,False,48687303954,sebastian.slesicki@kuba.pl
1,1,Filip,Maciejewski,M,60060840459,Wrocław,ul. Pułtuska 63/34,False,48210039472,re291kc2b9q4d4eu@opayq.com
2,2,Elena,Gamrat,K,74110877795,Szczecin,ul. Oskara Kolberga 94/2,False,48744588708,e-gamra74@wp.pl
3,3,Victoria,Kuklińska,K,90020124895,Wrocław,ul. Jana Pawła II 18/7,False,48068319561,l8xed0t9129@gmail.com
4,4,Paulina,Lechończak,K,93092394292,Świdnica,ul. Podgórna 14/9,False,48819095045,pau_lechonc8037@onet.pl
...,...,...,...,...,...,...,...,...,...,...
1455,1455,Józef,Gerlipp-Krajewski,M,86050432152,Tomaszów Mazowiecki,ul. Wiązowa 46/28,False,48757553740,gerlip.jozef@wp.pl
1456,1456,Maria,Smułkowska,K,60070730856,Wąchock,ul. Kolibrów 77/7,False,48426326948,smulkows.m@wp.pl
1457,1457,Krystian,Gumula,M,72120117856,Wrocław,ul. Strażacka 70/2,False,48042324185,kry-gumul1972@gmail.com
1458,1458,Maria,Ścisłowicz,K,None,Gdynia,ul. Różana 98/45,False,48524055862,marscislowic@o2.pl


In [20]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [21]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(74)

In [22]:
np.sum(pd.isna(clients_df["PESEL"]))

np.int64(253)

# auta

In [23]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [24]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [25]:
df_samochody = create_car_table(max(car_ids))

In [26]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Audi,A6 allroad,4,1997,SUV,False,szary,Diesel,5.1,320,Automatyczna,5,5,False
1,2,Ford,Focus,4,2011,SUV,False,niebieski,Benzyna,1.9,125,Automatyczna,5,5,False
2,3,BMW,X5,4,2000,Sedan,False,czarny,Diesel,5.0,306,Automatyczna,5,2,False
3,4,Nissan,Micra,4,2008,SUV,False,biały,Benzyna,1.1,80,Manualna,5,2,False
4,5,Volkswagen,Golf,4,2000,Kombi,True,niebieski,Diesel,1.7,110,Manualna,5,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,1076,Opel,Corsa,4,2007,SUV,False,srebrny,Benzyna,1.1,75,Manualna,5,5,True
1076,1077,Skoda,Fabia,4,2012,Sedan,True,szary,Benzyna,1.0,60,Manualna,5,5,False
1077,1078,Fiat,500,4,2000,SUV,False,czarny,Elektryczny/Benzyna,1.1,69,Manualna,5,5,True
1078,1079,Hero,Karizma 2014,2,2003,,False,czarny,Benzyna,0.7,92,Manualna,0,1,False


In [27]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [28]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Automatyczna', 'Manualna', 'Półautomatyczna'], dtype=object)

# Usługi

In [29]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [30]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [31]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [32]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_17260\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2008' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [33]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [34]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,2,2014-01-04,80800.0
1,2,2,2,3,2014-01-05,52800.0
2,3,3,3,3,2014-01-05,73300.0
3,4,4,4,2,2014-01-08,31800.0
4,5,5,5,2,2014-01-09,22000.0
...,...,...,...,...,...,...
743,744,1072,1451,3,2017-12-22,38800.0
744,745,1074,923,2,2017-12-28,21500.0
745,746,1076,1455,2,2017-12-29,24200.0
746,747,1077,1456,2,2017-12-29,21300.0


In [35]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [36]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [37]:
loyalty_card_customer_ids = [el[0] for el in loyalty_card_dates]
loyalty_card_dates_sep = [el[1] for el in loyalty_card_dates]

In [38]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:
        date_index = loyalty_card_customer_ids.index(customer_id)
        if loyalty_card_dates_sep[date_index] < df_sell.loc()[i]['data_zaksięgowania']:
            sell_prices[i] *= 0.97
            #print(i)

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
#print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_17260\1822833189.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:


In [39]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,12,21,2,47382.0,18,2014-01-22
1,2,8,22,3,145656.0,6,2014-01-22
2,3,9,23,2,34200.0,1,2014-01-22
3,4,10,24,3,31800.0,18,2014-01-23
4,5,11,25,2,65500.0,1,2014-01-24
...,...,...,...,...,...,...,...
736,737,135,1452,2,48984.0,12,2017-12-23
737,738,1072,1453,3,51000.0,6,2017-12-24
738,739,1071,592,2,22900.0,1,2017-12-26
739,740,1077,1457,2,32130.0,6,2017-12-30


In [40]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

np.int64(0)

In [41]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [42]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(31), array([47])), (np.int64(32), array([16])), (np.int64(35), array([ 7, 46])), (np.int64(38), array([20])), (np.int64(39), array([23, 60])), (np.int64(43), array([21])), (np.int64(45), array([24])), (np.int64(47), array([7])), (np.int64(44), array([4])), (np.int64(50), array([6])), (np.int64(54), array([20, 55])), (np.int64(55), array([28, 13])), (np.int64(56), array([35])), (np.int64(59), array([39])), (np.int64(61), array([51,  8])), (np.int64(62), array([41])), (np.int64(63), array([5])), (np.int64(66), array([21])), (np.int64(67), array([32])), (np.int64(68), array([45, 11])), (np.int64(70), array([31])), (np.int64(74), array([21])), (np.int64(77), array([5])), (np.int64(79), array([50])), (np.int64(80), array([25, 25])), (np.int64(81), array([6, 5])), (np.int64(82), array([ 3, 14])), (np.int64(78), array([ 9, 31])), (np.int64(89), array([ 4, 51])), (np.int64(92), array([39, 40, 25])), (np.int64(93), array([13])), (np.int64(94), array([12, 13])), (np.int64(98), array([

In [43]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [44]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Zawór EGR,,500,1,1
1,Głośnik,SEAT Arona,128,2,1
2,Zewnętrzna klamka drzwi,Volvo XC90,19,3,1
3,Tłumik,,200,3,1
4,Światło mgłowe,Ford Focus,43,4,1
...,...,...,...,...,...
729,Silnik,Nissan Qashqai 2.0,2821,498,1
730,Spojler,Opel Zafira złoty,664,499,1
731,Rozrusznik,Opel Zafira,703,499,1
732,Zawór EGR,,500,500,1


# Wyposażenie

In [45]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [46]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Zawór EGR,,500.0
1,2,Głośnik,SEAT Arona,128.0
2,3,Zewnętrzna klamka drzwi,Volvo XC90,19.0
3,4,Tłumik,,200.0
4,5,Światło mgłowe,Ford Focus,43.0
...,...,...,...,...
727,537,Deska rozdzielcza,BMW X6,415.0
728,538,Alternator,BMW X6,649.0
729,539,Silnik,Nissan Qashqai 2.0,2821.0
730,540,Spojler,Opel Zafira złoty,664.0


# Użyte części - uzupełnienie

In [47]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df['cena'] = pd.to_numeric(used_parts_df['cena'])

for i in range(len(used_parts_df)):
    used_parts_df.loc()[i, 'cena'] *= rng.uniform(0.9, 1.1)
    used_parts_df.loc()[i, 'cena'] = np.round(used_parts_df.loc()[i, 'cena'], 0)

used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość', 'cena']]

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_17260\1878107147.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '497.69761053254575' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  used_parts_df.loc()[i, 'cena'] *= rng.uniform(0.9, 1.1)


In [48]:
used_parts_df

,id,id_usługi,id_części,ilość,cena
0,1,1,1,1,498.0
1,2,2,2,1,127.0
2,3,3,3,1,19.0
3,4,3,4,1,207.0
4,5,4,5,1,41.0
...,...,...,...,...,...
729,730,498,539,1,2722.0
730,731,499,540,1,713.0
731,732,499,541,1,637.0
732,733,500,1,1,512.0


In [49]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [50]:
# def get_service_date(part_id):
    
#     index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
#     service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
#     return service_date
    

# buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
# parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

# for i in range(len(buy_parts_quantity)):
#     part_id = buy_parts_quantity['id_części'][i]
#     quantity = buy_parts_quantity['ilość'][i]
#     price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
#     total_cost = np.floor(price * rng.uniform(0.75, 0.95))[0]
#     service_date = get_service_date(part_id)
    
#     row = [int(i+1), int(part_id), total_cost, int(quantity)]
#     parts_purchase_pd.loc[i] = row


In [51]:
# parts_purchase_pd

In [52]:
# parts_purchase_pd.to_csv("outputs/zakup_części.csv", index=False)

In [53]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [54]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)

# Usługi uzupełnienie

In [55]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

for i in range(len(df_services)):
    if df_services.loc()[i]['id_klienta'] == 0:
        df_services.loc[i, 'cena'] = 0
    else:
        df_services.loc[i, 'cena'] = np.round(service_prices[i] * (rng.uniform(1.25, 1.7)), -1).astype(int)

In [56]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,31,59,6,2014-02-20,690.0
1,2,32,61,8,2014-02-20,190.0
2,3,35,66,6,2014-02-23,280.0
3,4,38,0,8,2014-02-27,0.0
4,5,39,73,6,2014-02-28,120.0
...,...,...,...,...,...,...
496,497,1073,1454,8,2017-12-26,1810.0
497,498,1075,653,4,2017-12-28,3860.0
498,499,781,1127,6,2017-12-29,1750.0
499,500,1079,1458,8,2017-12-31,630.0


In [57]:
df_services.to_csv("outputs/usługi.csv", index=False)